In [131]:
%load_ext autoreload
%autoreload 2

# import pgqc
from pgqc.ava import ava
from pgqc.nscluster import clusterBy_KmerJC, summarize_NSClusters
from pgqc.utils import parse_PresAbs_CSV_Roary, parse_PresAbs_CSV_Panaroo, get_PG_Stats_FromPresAbs
import screed
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [132]:
!ls -1 ../test_data_large/Roary_Default_AllIsolates

gene_presence_absence.Rtab
gene_presence_absence.csv
pan_genome_reference.ava.tsv
pan_genome_reference.fa
pg.ref.ava.tsv
summary_statistics.txt


In [133]:
# Roary out files 
Roary_Default_OutDir = "../test_data_large/Roary_Default_AllIsolates"

Roary_PG_Ref_FA = f"{Roary_Default_OutDir}/pan_genome_reference.fa"
Roary_PresAbs_CSV = f"{Roary_Default_OutDir}/gene_presence_absence.csv"
Roary_AvA_TSV = f"{Roary_Default_OutDir}/pan_genome_reference.ava.tsv"

# Panaroo out files
Panaroo_Default_OutDir = "../test_data_large/Panaroo_Moderate_MergeParalogs_AllIsolates"

Panaroo_PG_Ref_FA = f"{Panaroo_Default_OutDir}/pan_genome_reference.fa"
Panaroo_PresAbs_CSV = f"{Panaroo_Default_OutDir}/gene_presence_absence.csv"
Panaroo_AvA_TSV = f"{Panaroo_Default_OutDir}/pan_genome_reference.ava.tsv"



In [134]:
filename = Roary_PG_Ref_FA

NumParsedRecords = 0

for record in screed.open(filename):

    FullName = record.name

    ShortName = record.name.split(" ")[-1]
    NumParsedRecords += 1
    sequence = record.sequence

    print(FullName, "  ", ShortName)

    if NumParsedRecords > 10: break
    

N0072_00005 dnaA    dnaA
N0072_00010 dnaN    dnaN
N0072_00015 recF    recF
N0072_00020 group_3644    group_3644
N0072_00025 gyrB    gyrB
N0072_00030 gyrA    gyrA
N0072_00035 group_2302    group_2302
N0072_00040 putative conserved membrane protein    protein
N0072_00055 group_2303    group_2303
N0072_00060 group_2304    group_2304
N0072_00065 cell wall synthesis protein CwsA    CwsA


## Panaroo processing

In [135]:
# Panaroo_PG_AvA_DF = ava(Panaroo_PG_Ref_FA, 31)
# Panaroo_PG_AvA_DF.to_csv(Panaroo_AvA_TSV, sep = "\t", index=False)
# Panaroo_PG_AvA_DF.shape 

In [136]:
Panaroo_PG_AvA_DF = pd.read_csv(Panaroo_AvA_TSV, sep = "\t")
Panaroo_PG_AvA_DF.shape

(1502, 5)

In [137]:
Panaroo_PresAbs_DF = parse_PresAbs_CSV_Panaroo(Panaroo_PresAbs_CSV)

print(Panaroo_PG_AvA_DF.shape, Panaroo_PresAbs_DF.shape) 

/Users/mgmarin/Documents/FarhatLab/GitRepos/pg-ava-kcomp/pgqc/utils.py:13: DtypeWarning: Columns (6,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  i_Gene_PresAbs_DF.columns = [ x.split(".Bakta")[0] for x in i_Gene_PresAbs_DF.columns ]
/Users/mgmarin/Documents/FarhatLab/GitRepos/pg-ava-kcomp/pgqc/utils.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return i_Gene_PresAbs_DF


(1502, 5) (4292, 160)


/Users/mgmarin/Documents/FarhatLab/GitRepos/pg-ava-kcomp/pgqc/utils.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [138]:
Panaroo_PresAbs_DF.head(4)

,Gene,N0072,N0153,DNA028,N0155,N0145,DNA075,M0003941_3,DNA019_Vash,DNA091,...,MFS-54,MFS-58,MFS-51,MFS-42,MFS-52,MFS-56,MFS-60,MFS-61,MFS-181,NumAsm_WiGene
Gene,,,,,,,,,,,,,,,,,,,,,
group_3,group_3,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
group_0,group_0,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
mmr,mmr,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
group_11,group_11,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158


In [139]:
#Panaroo_PresAbs_DF.columns

## Roary processing

In [140]:
# Roary_PG_AvA_DF = ava(Roary_PG_Ref_FA, 31)
# Roary_PG_AvA_DF.to_csv(Roary_AvA_TSV, sep = "\t", index=False)
# Roary_PG_AvA_DF.shape


In [141]:
Roary_PG_AvA_DF = pd.read_csv(Roary_AvA_TSV, sep = "\t")
Roary_PG_AvA_DF.shape

(7175, 5)

In [142]:
Roary_PresAbs_DF = parse_PresAbs_CSV_Roary(Roary_PresAbs_CSV)

print(Roary_PG_AvA_DF.shape , Roary_PresAbs_DF.shape)

/Users/mgmarin/Documents/FarhatLab/GitRepos/pg-ava-kcomp/pgqc/utils.py:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  


(7175, 5) (5417, 160)


/Users/mgmarin/Documents/FarhatLab/GitRepos/pg-ava-kcomp/pgqc/utils.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [143]:
Roary_PresAbs_DF.head(2)

,Gene,01_R1134,01_R1430,02_R0894,02_R1179,02_R1708,02_R1896,18_0621851,3003-06,4549-04,...,mada_129,mada_139,mada_151,mada_2-1,mada_2-25,mada_2-31,mada_2-42,mada_2-46,mada_2-53,NumAsm_WiGene
Gene,,,,,,,,,,,,,,,,,,,,,
TetR/AcrR family transcriptional regulator,TetR/AcrR family transcriptional regulator,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
group_989,group_989,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158


In [144]:
Roary_PresAbs_NSC_Filt_DF, Roary_GraphInfoDict = clusterBy_KmerJC(Roary_PG_AvA_DF, Roary_PresAbs_DF, 0.8)
Roary_PresAbs_NSC_Filt_DF.shape  

(4485, 160)

In [145]:
Roary_PresAbs_NSC_Filt_DF.head(4)

,Gene,01_R1134,01_R1430,02_R0894,02_R1179,02_R1708,02_R1896,18_0621851,3003-06,4549-04,...,mada_129,mada_139,mada_151,mada_2-1,mada_2-25,mada_2-31,mada_2-42,mada_2-46,mada_2-53,NumAsm_WiGene
4424,NS_350_group_448---group_449,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
4428,NS_66_group_519---kdpE---kdpD---group_521,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
4429,NS_187_smc---group_1516,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158
4430,NS_199_group_1408---group_1407,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,158


In [146]:
get_PG_Stats_FromPresAbs(Roary_PresAbs_DF, 158)    

Accessory Thresh: 156.42
# of core genes: 3492
# of accessory genes: 1925


(5417, 3492, 1925)

In [147]:
get_PG_Stats_FromPresAbs(Roary_PresAbs_NSC_Filt_DF, 158)

Accessory Thresh: 156.42
# of core genes: 3695
# of accessory genes: 790


(4485, 3695, 790)

In [148]:
Panaroo_PresAbs_NSC_Filt_DF, Panaroo_GraphInfoDict = clusterBy_KmerJC(Panaroo_PG_AvA_DF, Panaroo_PresAbs_DF, 0.8)
Panaroo_PresAbs_NSC_Filt_DF.shape

(4127, 160)

In [149]:

Panaroo_CoreGenes_List = Panaroo_PresAbs_DF.query("NumAsm_WiGene >= 157")["Gene"].values
Panaroo_Filt08_Cluster_WiType_DF, Panaroo_Filt08_ClusterType_Count = summarize_NSClusters(Panaroo_GraphInfoDict["Filt_Cluster_DF"],
                                                                                          Panaroo_CoreGenes_List)

# NucSim Clusters: 124
# of total clustered genes: 289
# of NS clusters w/ only Core genes: 5
# of NS clusters w/ only Accessory genes: 29
# of NS clusters w/ BOTH Core & Accessory genes: 90


In [150]:
Roary_CoreGenes_List = Roary_PresAbs_DF.query("NumAsm_WiGene >= 157")["Gene"].values
Roary_Filt08_Cluster_WiType_DF, Roary_Filt08_ClusterType_Count = summarize_NSClusters(Roary_GraphInfoDict["Filt_Cluster_DF"],
                                                                                      Roary_CoreGenes_List)

# NucSim Clusters: 542
# of total clustered genes: 1555
# of NS clusters w/ only Core genes: 8
# of NS clusters w/ only Accessory genes: 325
# of NS clusters w/ BOTH Core & Accessory genes: 209
